<a href="https://colab.research.google.com/github/martinahuang/CUS615/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is part the of **Dr. Christoforos Christoforou's** course materials. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials or any of their derivatives without the instructor's express written consent.

# Final Exam Project
**Professor**: Dr. Christoforos Christoforou

The objective of this project is to reproduce the data analysis method presented in the paper:

* **"Mining Big Data to Extract Patterns and Predict Real-Life Outcomes"**
by Michal Kosinski, Yilun Wang, Himabindu Lakkaraju, and Jure Leskovec.


You must implement the analysis using python code. This notebook provides you with the starting code to load the data and guides you through the requirement and each sub-task of the analysis. To complete this project you are expected to:

1. Carefully read the paper.
2. Refer the class lecture where this paper was discussed in detail.
3. Research various python libraries and utility functions needed to implement your analysis.
4. Complete all the challenges outlined in this notebook.



## Student Information 
Complete your information in the form provided below. This is visible in colab. 


In [0]:
#@title Student Information 
student_name = "Martina Huang" #@param {type:"string"}
course_code = "CUS 615 -CRN: 12094" #@param {type:"string"}


## Starter code - Configuring the analysis environment
The code cells below provide you a starting code to setup the environment for you to complete this project's tasks.

In particular, the code downloads the three dataset associated with this project and stores them in the folder `/content/sample_data/`. The data files from the paper comprise the `likes.csv` file, the `users.csv` file and the `users-likes.csv`; description of each data file is availabile in the paper.




In [109]:
#
# Run this sell to download the data for your analysis. 
#
!wget -O /content/sample_data/sample_dataset.zip https://osf.io/9m87k/download
!unzip /content/sample_data/sample_dataset.zip -d /content/sample_data/


--2020-05-05 05:00:32--  https://osf.io/9m87k/download
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/fuqjg/providers/osfstorage/5e946850f1353503a3d52f24?action=download&direct&version=1 [following]
--2020-05-05 05:00:33--  https://files.osf.io/v1/resources/fuqjg/providers/osfstorage/5e946850f1353503a3d52f24?action=download&direct&version=1
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314322145 (300M) [application/octet-stream]
Saving to: ‘/content/sample_data/sample_dataset.zip’

/content/sample_dat 100%[===================>] 299.76M  14.5MB/s    in 21s     

2020-05-05 05:00:55 (14.1 MB/s) - ‘/content/sample_data/sample_dataset.zip’ saved [314322145/314322145]

Archive:  /content/sample_data/sa

In [110]:
#
# Install additional package not available in colab.
#
pip install factor_analyzer

In [0]:
#
# You will likely need the following python packages when implementing this project.
# Add any additional package as you deem necessary 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs
from factor_analyzer import Rotator

# Additional package you want to use:
from sklearn.model_selection import train_test_split


## Challenge 1 : Load the data into dataframes
In this challenge you are expected to load the data from the raw files into the following variables:
-  `users` : should reference a dataframe with the data in the `users.csv` file
-  `likes` : should reference a dataframe with the data in the `likes.csv` file
-  `ul` : should reference a dataframe with the data in `users.csv` file

Then print the top 10 rows of reach dataframe.

In [0]:
# Your code goes here 
# 

datafile1 = './sample_data/likes.csv'  
datafile2 =  './sample_data/users.csv'  
datafile3 = './sample_data/users-likes.csv'

likes = pd.read_csv(datafile1)
users = pd.read_csv(datafile2)
ul = pd.read_csv(datafile3)


In [113]:
# Answer summary cell: Print the top 10 row from each datafame. 
# 
likes.head(10)

,likeid,name
0,3c1636c878e6eb2acfd00c6b61086e38,REIGN by Paul Gibson
1,feca46ddb8ef04f86172ace0cb7e004c,Cupcake Wishes & Birthday Dreams
2,b65f46d64c688fe98bdbcf93a76a71fc,Yo también me rei de la caída de otro jejeje
3,9c5c8bb82d2cd46fbd7582f944fe370e,Abraham Joshua Heschel Day School- Alumni Network
4,2d82fa84ad79b085dc516dde154327a2,Kennesaw Farmer's Market
5,0a7a01c82143347fc4703faa4e7f415c,Karlsruher SC
6,8ed466215d9a6a4168120387d6e390b8,Intercessory Prayer
7,f82d6a5b5fbfd48bd293971f998d4da5,Nitti Development
8,d2c160d8e526a8cd38a73b3186a06fbf,Lakewood Organic Juices
9,5485abaac8afc3f3807a92515aca3f2a,Space Lieutenant Z Hippo Tomahawk


In [114]:
users.head(10)

,userid,gender,age,political,ope,con,ext,agr,neu
0,54f34605aebd63f7680e37ffd299af79,0,33,0.0,1.26,1.65,1.17,-1.76,0.61
1,86399f8c44ba54224b2e60177ca89fa9,1,35,0.0,1.07,0.17,-0.14,1.49,0.30
2,84fab50f3c60d1fdc83aa91b5e584a78,1,36,0.0,0.89,1.28,0.86,1.07,0.99
3,f3b8fdaccce12ef6352bfad4d6052fe9,0,39,NaN,0.33,-1.01,-0.33,-0.68,0.92
4,8b06ea5e9cb87c61da387995450607f7,0,31,NaN,0.15,0.47,1.17,-1.01,-0.32
5,7a2ec9c1de4ec137367e66ad759ec848,0,38,NaN,1.26,0.76,-0.46,-1.76,-0.76
6,259f8f9cce95b214dc9924af48bcd6f0,1,36,NaN,1.44,-0.12,-0.21,0.99,0.18
7,ea3d6b771a87038cbc9e7da666f82b7a,0,58,NaN,0.70,-0.94,0.86,0.99,-0.32
8,e9552c9fe95d5cda02e2b55c05a943bc,0,37,NaN,1.63,0.32,0.98,1.49,-1.50
9,db85e9a0b1ab33ab7be878b9c236e7f2,1,33,0.0,0.70,0.54,-0.27,-0.26,0.92


In [115]:
ul.head(10)

,userid,likeid
0,71bc7c0901488aec6d30f0add257e7c5,3c1636c878e6eb2acfd00c6b61086e38
1,978ab8e90c4d6ad1a48ef5c973b62f4d,feca46ddb8ef04f86172ace0cb7e004c
2,85123b0e358907725cf19a2cb0ec3983,b65f46d64c688fe98bdbcf93a76a71fc
3,ce110562b3e2f7e5cad3775b32d9caa5,b65f46d64c688fe98bdbcf93a76a71fc
4,8188d20745471273fa69ba44a5b28473,b65f46d64c688fe98bdbcf93a76a71fc
5,e9c621f322640e5f3b3842d19b38aaa8,b65f46d64c688fe98bdbcf93a76a71fc
6,f0c99cfcdbda49dc424848666c6b31a9,b65f46d64c688fe98bdbcf93a76a71fc
7,7de7b4609c6b51d956d27050c958d885,b65f46d64c688fe98bdbcf93a76a71fc
8,05ce4ab49d90139ec1f7af9f0255c9c6,b65f46d64c688fe98bdbcf93a76a71fc
9,051caa763b05e64d4c8699078e942dfd,9c5c8bb82d2cd46fbd7582f944fe370e


## Challenge 2: Create the sparse user-likes matrix 
Create a **sparse matrix** that stores the data for the user-likes associations as described in the paper. The name of the user-likes matrix must be `M`, to keep the naming convention used in the paper. It is important that to store the data in a sparse matrix; you might need to research the topic of using sparse matrices in python (i.e. look at the `scipy.sparse` package and the `csr_matrix` class). *Hint: For this challenge,  you might find  the `merge` method from `pandas` library useful*. 

Once you create the matrix print its shape on the answer summary cell.

In [0]:
#Merge the files first then matrix
# Your code goes here.
# 

likes['like_row'] = range(len(likes))
users['user_row'] = range(len(users))


In [0]:
ul1 = pd.merge(ul, likes[['likeid','like_row']] , on = 'likeid', how = 'left')
df = pd.merge (ul1, users [['userid','user_row']] , on = 'userid', how = 'left')
df['id']=range(len(df))

In [118]:
df.head()

,userid,likeid,like_row,user_row,id
0,71bc7c0901488aec6d30f0add257e7c5,3c1636c878e6eb2acfd00c6b61086e38,0,59352,0
1,978ab8e90c4d6ad1a48ef5c973b62f4d,feca46ddb8ef04f86172ace0cb7e004c,1,36792,1
2,85123b0e358907725cf19a2cb0ec3983,b65f46d64c688fe98bdbcf93a76a71fc,2,30997,2
3,ce110562b3e2f7e5cad3775b32d9caa5,b65f46d64c688fe98bdbcf93a76a71fc,2,50636,3
4,8188d20745471273fa69ba44a5b28473,b65f46d64c688fe98bdbcf93a76a71fc,2,72223,4


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10612326 entries, 0 to 10612325
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   userid    object
 1   likeid    object
 2   like_row  int64 
 3   user_row  int64 
 4   id        int64 
dtypes: int64(3), object(2)
memory usage: 485.8+ MB


In [0]:
#M = csr_matrix (data, (rows,columns))

data = np.array(df['id']).astype(int)
row = np.array(df['user_row']).astype(int)
col = np.array(df['like_row']).astype(int)

M =csr_matrix((data, (row,col)),dtype=int)

In [121]:
M.shape

(110728, 1580284)

In [91]:
num_rows, num_cols = M.shape
print (num_rows, num_cols)

110728 1580284


In [0]:
#
# Answer summary cell:  Print the shape of the matrix M
# The shape is 110,728 x 1,580,284


array([ 59352,  36792,  30997, ...,  44279,  47598, 103984])

## Challege 3: Trim the sparse user-likes matrix M.
The paper calls for the matrix `M` to be trimmed (i.e. to remove users which did not like a minimum number of posts and posts that did not receive a minimum number of likes). In this challenge, you are expected to implement the trimming preprocessing step as described in the paper. The resulting matrix must be a sparse matrix. *Hint: you might want to research to index slice sparse matrices in python*

In [0]:
#
# Your code here. 
# rowsums (likes per user , 'like_row') > 50, colsums (users per like 'user_row')>=150
#np.delete
import numpy.ma as ma

#rowsums = row[(-100<=row)&(row<=100)]

#data = np.array(df['id']).astype(int)
#row = np.array(df['user_row']).astype(int)
#col = np.array(df['like_row']).astype(int)

#M =csr_matrix((data, (row,col)),dtype=int)

#row_sums = row[(50<=row)]
#col_sums = col[(150<=col)]

#row_sums = row
#sum(i > 50 for i in row_sums)

#>>> j = [4, 5, 6, 7, 1, 3, 7, 5]
#>>> sum(i > 5 for i in j)

#print(len(col_sums))
#print(max(col_sums))

In [0]:
M1 =csr_matrix((M, (row_sums,col_sums)),dtype=int)

In [0]:
TM =csr_matrix((M, (row_sums,col_sums)),dtype=int)

In [0]:
#
# Answer summary cell:  Print the shape of the matrix M after trimming. 
# 

1


## Challenge 4:  Split the sparse matrix into a training and testing matrices
To build a prediction model the paper calls for separating the data matrix (i.e. the user-like matrix) into a training and testing matrices. As part of this challenge, you need to create a training matrix `M_train` that includes the preferences of 80% of the users, and a testing matrix `M_test` that include the preferences of the remaining 20% of the users in the dataset. Both matrices `M_train` and `M_test` must be stored as sparse matrices. User will be randomly assigned to the training or testing set.

Print the shapes of the `M_train` and `M_test` matrices.




In [128]:
#X = 
#y =

# Your code here. 
M_test, M_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

NameError: ignored

In [0]:
#
# Answer summary cell: Print the shapes of M_train, M_test and M matrices.  
# 

## Challege 5 : Reduce the dimensionality of the M_train matrix using SVD

In this challenge, you must use singular value decomposition (SVD) to reduce the dimensionality of the training matrix `M_train`. Remember that `M_train` is encoded as a sparse matrix so you would need to use an appropriate implementation of the SVD for sparse matrices. Your output should be the matrices `U`, `S` and `Vt` that correspond to the left singular vectors, the singular values, and the right singular vectors, respectively. You must specify a parameter `k` that defines the number of reduced dimensions you want to keep.

Print the shapes of the `U`, `S` and `Vt` matrices and the value you selected for parameter `k`.



In [0]:
#
# Your code here. 
#

In [0]:
#
# Answer summary cell: Print the shapes of U, S and Vt; and the value you selected for parameter `k`.
# 

## Challege 6 : Use the varimax algorithm to rotate the SVD dimensions. 
Use the varimax algorithm to rotate the SVD dimensions as described in the paper. A good python library that implements varimax algorithm is the `factor_analyzer` and its `Rotator` class (already installed in this notebook for your in the started cell above); i.e. using `rotator = Rotator(method='varimax')`. Store the varimax SVD into the matrices `U_rot` and `V_rot` respectively. *Hint: you can apply varimax of `Vt` to get `V_rot` and then multiply `M` by  `Vt_rot` to obtain the `U_rot` (as done in the paper); you might want to research hot to user the `Rotator` class*. 

Print the shapes of the matrices `U_rot` and `V_rot`.


In [0]:
#
# Your code here. 
#


In [0]:
#
# Answer summary cell: Print the shapes of the matrices `U_rot` and `V_rot`
# 

## Challenge 7 : Train two predictive model to predict the political  affilication of users.
Build a predictive model that predicts the political affiliation of the users given the user's post preferences (as those are captured in the paper's dataset). To train your model you can use data from matrix `M_train` only. You are expected to use two different predictive models of your choice (i.e. Knn, SVM, Logistic Regression or some other model of your choice).

As part of this challenge you are expected to:
- Apply any necessary preprocessing to handle missing values.
- The models must be applied on the reduced-dimensional feature generated by your analysis above (i.e. use the singular rotated singular dimensions)
- Use cross-validation for model selection (i.e. to identify optimal parameters for your model, or any meta-parameters)
- Report training and cross-validation performance of the two models as well as the optimal parameters for your model identified by cross validation. 


**FIRST MODEL TRAINING: Use the cells below to train the predictive model using the first model you selected.**

In [0]:
#
# Use this cell to train the first model you selected for challenge 7. 
# Add additional cells as needed below this cell.
#


**FIRST MODEL ACCURACY REPORT: Use the cell below to report on the accuracy of your first model**


In [0]:
#
# Use this cell to report on the accuracy of the first model (i.e. training and cross validation performance)
# and model selection results. 


**SECOND MODEL TRAINING: Use the cells below to train the predictive model using the second model you selected.**

In [0]:
#
# Use this cell to train the second model you selected for challenge 7. 
# Add additional cells as needed below this cell.
#


**SECOND MODEL ACCURACY REPORT: Use the cell below to report on the accuracy of your second model**

In [0]:
#
# Use this cell to report on the accuracy of the second model (i.e. training and cross validation performance)
# and model selection results.


## Challenge 8 Apply your model in the independent test set.
User the cells below to apply your two predictive models on the test dataset `M_test` and report testing accuracy for each model.

As part of this challenge you are expected to:
- Project each observation in the test dataset `M_test` onto the reduced-dimensional features space.
- Apply each on of your model to make predict each user's political preference (for users in the testing set only)
- Report testing accuracy on each of your models.
- Make an assessment as to which model is better and whether either of your models is overfitted or under-fitted.

In [0]:
#
# Use this cell to comlete challege 8; add more cells as necessary. 
#

# END OF EXAM/CODE


*Copyright Statement*: Copyright © 2020 Christoforou. The materials provided by the instructor of this course, including this notebook, are for the use of the students enrolled in the course. Materials are presented in an educational context for personal use and study and should not be shared, distributed, disseminated or sold in print — or digitally — outside the course without permission. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials  as well as any of their derivatives without the instructor's express written consent.